In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from tqdm import tqdm

import keras

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold

import cv2
import os

Using TensorFlow backend.


In [31]:
df_train = pd.read_csv('Data/labels.csv')
df_train.head()

,Unnamed: 0,id,breed
0,0,n02108551_978,Tibetan_mastiff
1,1,n02112350_223,keeshond
2,2,n02102480_4564,Sussex_spaniel
3,3,n02109525_18948,Saint_Bernard
4,4,n02096437_3195,Dandie_Dinmont


In [3]:
#df = df.groupby('domain')['ID'].nunique()
df = df_train.groupby('breed').nunique()
df = df.sort_values('id', ascending=False)
df

,Unnamed: 0,id,breed
breed,,,
Maltese_dog,252,252,1
Afghan_hound,239,239,1
Scottish_deerhound,232,232,1
Pomeranian,219,219,1
Samoyed,218,218,1
Bernese_mountain_dog,218,218,1
Irish_wolfhound,218,218,1
Shih-Tzu,214,214,1
Great_Pyrenees,213,213,1


In [26]:
train, test = train_test_split(df_train, test_size=0.1, random_state=1)#, stratify=y_train_raw)

In [30]:
train

,breed
11373,otterhound
15062,Irish_terrier
6256,Weimaraner
15043,Ibizan_hound
6857,affenpinscher
18077,Airedale
9242,standard_poodle
14125,English_setter
16166,basenji
19630,Japanese_spaniel


In [33]:
df = train.groupby('breed').nunique()
df

,breed
breed,
Afghan_hound,1
African_hunting_dog,1
Airedale,1
American_Staffordshire_terrier,1
Appenzeller,1
Australian_terrier,1
Bedlington_terrier,1
Bernese_mountain_dog,1
Blenheim_spaniel,1


In [14]:
y_train = y_train.nunique()

AttributeError: 'int' object has no attribute 'nunique'

In [10]:
X_test.shape

(2058,)

In [ ]:
i = 0 
for _, f, breed in tqdm(df_train.values):
    file = f.split('_')[0] + '-' + breed
    path = 'Data/Images/{}/{}.jpg'.format(file,f)
    #print(path)
    img = cv2.imread(path)
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (img_size, img_size)))
    y_train.append(label)
    i += 1

In [5]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255

NameError: name 'y_train' is not defined